In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import numpy as np

import jieba
import re

from sklearn.model_selection import train_test_split

In [2]:
num_examples = 10000
BATCH_SIZE = 128
embedding_dim = 256
units = 1024
EPOCHS = 20

## 加载数据并向量化

In [3]:
data_path = 'cmn.txt'
df = pd.read_table(data_path,header=None)
df.columns=['inputs', 'targets', 'others']
df = df[['inputs', 'targets']]
df.head()

,inputs,targets
0,Hi.,嗨。
1,Hi.,你好。
2,Run.,你用跑的。
3,Wait!,等等！
4,Wait!,等一下！


In [4]:
def preprocess_sentence(w):
    w = '<start> ' + ' '.join(jieba.cut(w)).lower() + ' <end>'
    w = re.sub(r'[" "]+', " ", w)
    return w 

def create_dataset(data, num_examples=None):
    eng = [preprocess_sentence(w) for w in df.inputs.values.tolist()[:num_examples]]
    cha = [preprocess_sentence(w) for w in df.targets.values.tolist()[:num_examples]]
    return eng, cha

In [5]:
en, sp = create_dataset(df, 5432)
print(en[-1])
print(sp[-1])

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ZCF\AppData\Local\Temp\jieba.cache
Loading model cost 1.064 seconds.
Prefix dict has been built successfully.


<start> he wants a book to read . <end>
<start> 他 想 找 本書來 讀 。 <end>


In [6]:
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

In [7]:
def load_dataset(data, num_examples=None):
    # 创建清理过的输入输出对
    inp_lang, targ_lang = create_dataset(data, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [8]:
# 尝试实验不同大小的数据集
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset("./cmn.txt", num_examples)

# 计算目标张量的最大长度 （max_length）
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

# 采用 80 - 20 的比例切分训练集和验证集
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# 显示长度
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

8000 8000 2000 2000


In [9]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
27 ----> don
5 ----> '
13 ----> t
41 ----> be
2412 ----> disappointed
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
72 ----> 不要
1456 ----> 失望
3 ----> 。
2 ----> <end>


## 创建一个 tf.data 数据集

In [10]:
BUFFER_SIZE = len(input_tensor_train)
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE

vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 15]), TensorShape([128, 15]))

## 编写编码器 （encoder） 和解码器 （decoder） 模型

In [11]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
    
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# 样本输入
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (128, 15, 1024)
Encoder Hidden state shape: (batch size, units) (128, 1024)


In [12]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # 隐藏层的形状 == （批大小，隐藏层大小）
        # hidden_with_time_axis 的形状 == （批大小，1，隐藏层大小）
        # 这样做是为了执行加法以计算分数  
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # 分数的形状 == （批大小，最大长度，1）
        # 我们在最后一个轴上得到 1， 因为我们把分数应用于 self.V
        # 在应用 self.V 之前，张量的形状是（批大小，最大长度，单位）
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # 注意力权重 （attention_weights） 的形状 == （批大小，最大长度，1）
        attention_weights = tf.nn.softmax(score, axis=1)

        # 上下文向量 （context_vector） 求和之后的形状 == （批大小，隐藏层大小）
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights
    
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (128, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (128, 15, 1)


In [13]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # 用于注意力
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # 编码器输出 （enc_output） 的形状 == （批大小，最大长度，隐藏层大小）
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x 在通过嵌入层后的形状 == （批大小，1，嵌入维度）
        x = self.embedding(x)

        # x 在拼接 （concatenation） 后的形状 == （批大小，1，嵌入维度 + 隐藏层大小）
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # 将合并后的向量传送到 GRU
        output, state = self.gru(x)

        # 输出的形状 == （批大小 * 1，隐藏层大小）
        output = tf.reshape(output, (-1, output.shape[2]))

        # 输出的形状 == （批大小，vocab）
        x = self.fc(output)

        return x, state, attention_weights
    
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (128, 6630)


## 定义优化器和损失函数

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

## 检查点（基于对象保存）

In [15]:
import os

checkpoint_dir = './training_checkpoints_4_1'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [16]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # 教师强制 - 将目标词作为下一个输入
        for t in range(1, targ.shape[1]):
            # 将编码器输出 （enc_output） 传送至解码器
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # 使用教师强制
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

## 训练

In [17]:
# 恢复检查点目录 （checkpoint_dir） 中最新的检查点
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [33]:
import time

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 40 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
    # 每 2 个周期（epoch），保存（检查点）一次模型
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.7628
Epoch 1 Batch 40 Loss 2.2956
Epoch 1 Loss 2.4123
Time taken for 1 epoch 304.1918475627899 sec

Epoch 2 Batch 0 Loss 2.0240
Epoch 2 Batch 40 Loss 1.9135
Epoch 2 Loss 1.9583
Time taken for 1 epoch 288.0244393348694 sec

Epoch 3 Batch 0 Loss 1.7051
Epoch 3 Batch 40 Loss 1.7726
Epoch 3 Loss 1.7902
Time taken for 1 epoch 286.6007413864136 sec

Epoch 4 Batch 0 Loss 1.6579
Epoch 4 Batch 40 Loss 1.6039
Epoch 4 Loss 1.6277
Time taken for 1 epoch 291.56291818618774 sec

Epoch 5 Batch 0 Loss 1.4373
Epoch 5 Batch 40 Loss 1.4115
Epoch 5 Loss 1.4969
Time taken for 1 epoch 289.05738139152527 sec

Epoch 6 Batch 0 Loss 1.2863
Epoch 6 Batch 40 Loss 1.3289
Epoch 6 Loss 1.3825
Time taken for 1 epoch 287.7144830226898 sec

Epoch 7 Batch 0 Loss 1.2857
Epoch 7 Batch 40 Loss 1.3462
Epoch 7 Loss 1.2794
Time taken for 1 epoch 284.82519578933716 sec

Epoch 8 Batch 0 Loss 1.1876
Epoch 8 Batch 40 Loss 1.1288
Epoch 8 Loss 1.1712
Time taken for 1 epoch 281.07025957107544 sec

Epoch 9 Batc

## 翻译

In [18]:
def evaluate(inputs):
    
    sentence = ' '.join([inp_lang.index_word[t] for t in inputs if t!=0])
    sentence = sentence[7:-5].strip()
    
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence

        # 预测的 ID 被输送回模型
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [19]:
def translate(inputs):
    result, sentence = evaluate(inputs)

    print(sentence)
    print('predict : ',re.sub(r'[" "]+', "", result[:-6]))

## 训练集

In [21]:
index = 911
for i in range(index, index+20):
    translate(input_tensor_train[i])
    target = [targ_lang.index_word[t] for t in target_tensor_train[i] if t>0]
    target = "".join(target)
    print(f'real : {target[7:-5]}\n')

it isn ' t too late for you .
predict :  这不跟你来说也沒有耐心。
real : 对你来说还不太晚。

come and dance with me .
predict :  过来和我跳舞啊!
real : 过来和我跳舞啊!

that boy is running .
predict :  这个男孩子在跑步。
real : 这个男孩子在跑步。

can you lend me 10 , 000 yen ?
predict :  你能借給我10000日元？
real : 你能借給我10000日元？

he has two daughters .
predict :  他有两个女儿。
real : 他有两个女儿。

you are not a coward .
predict :  你不是个懦夫。
real : 您不是个懦夫。

i ' ve been wanting to see you .
predict :  我一直想見你。
real : 我一直想見你。

we have to go by the rules .
predict :  我們必須離開了。
real : 我们必须遵守规则。

she gave us a vague answer .
predict :  她給了我們一個模糊的答案。
real : 她給了我們一個模糊的答案。

why worry about tom ?
predict :  為什麼要擔心tom？
real : 為什麼要擔心tom？

let ' s go !
predict :  我們走吧!
real : 走吧。

people from madrid are weird .
predict :  从马德里来的人都不會人。
real : 从马德里来的人都很怪。

tom will probably be there .
predict :  湯姆可能會去那裡。
real : 湯姆可能會去那裡。

don ' t stand near me .
predict :  不要站在我身邊。
real : 不要站在我身邊。

i ' m 17 , too .
predict :  我也紧张。
real : 我也是17岁。

i ' ll make you happy .
predict :  我會让你逃跑

## 测试集

In [23]:
index = 911
for i in range(index, index+20):
    translate(input_tensor_val[i])
    target = [targ_lang.index_word[t] for t in target_tensor_val[i] if t>0]
    target = "".join(target)
    print(f'real : {target[7:-5]}\n')

my stomach ' s full .
predict :  我的胃是脹滿的。
real : 我的胃是脹滿的。

i don ' t fear death .
predict :  我不怕死。
real : 我不怕死。

it sounds like a good idea .
predict :  它是个好主意。
real : 它聽起來是個好主意。

tom loves you .
predict :  汤姆关心你。
real : 汤姆爱你。

i miss you so much .
predict :  我如此想念你。
real : 我如此想念你。

what ' s your name ?
predict :  你叫什么名字？
real : 叫什麼名字?

look , she said .
predict :  “看啊！”她说道。
real : “看啊！”她说道。

don ' t shout .
predict :  不许大叫。
real : 不许大叫。

i would like your picture .
predict :  我想要你的照片。
real : 我想要你的照片。

i ' ve quit drinking beer .
predict :  我已經不喝啤酒了。
real : 我已經不喝啤酒了。

say it clearly .
predict :  說清楚。
real : 說清楚。

i don ' t want to play anymore .
predict :  我不想再玩。
real : 我不想再玩了。

tom doesn ' t think so .
predict :  汤姆可不这么认为。
real : 汤姆可不这么认为。

may i go now ?
predict :  我现在能去了吗？
real : 我现在能去了吗？

i met your father yesterday .
predict :  我昨天到湯姆在睡覺。
real : 昨天我見到了你父親。

nothing is worse than war .
predict :  没有什么比战争更糟的。
real : 没有什么比战争更糟的了。

you look like a monkey .
predict :  你看着像猴。
real : 你看着